In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2
scikit-learn-intelex==2021.20210714.120553


In [11]:
import pickle
import os
import pandas as pd
import numpy as np

In [2]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [3]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:

from pathlib import Path

PARENT_PATH = Path(os.getcwd()).parent.parent.parent
DATA_PATH = os.path.join(PARENT_PATH, 'data')

print(DATA_PATH)

/home/david/Elvis/data


In [8]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173,82,None,B00021,10.666667
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173,56,None,B00021,14.566667
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82,129,None,B00021,7.950000
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,-1,225,None,B00037,13.800000
5,B00037,2021-02-01 00:00:37,2021-02-01 00:09:35,-1,61,None,B00037,8.966667


In [9]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [12]:
np.mean(y_pred)

16.191691679979066

In [13]:
year = pd.DatetimeIndex(df['pickup_datetime']).year
month = pd.DatetimeIndex(df['pickup_datetime']).month

year = year.astype('string')
month = month.astype('string')

df['ride_id'] = f'2021/02_' + df.index.astype('str')

In [14]:
df['prediction'] = y_pred
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration,ride_id,prediction
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173,82,None,B00021,10.666667,2021/02_1,14.539865
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173,56,None,B00021,14.566667,2021/02_2,13.740422
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82,129,None,B00021,7.950000,2021/02_3,15.593339
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,-1,225,None,B00037,13.800000,2021/02_4,15.188118
5,B00037,2021-02-01 00:00:37,2021-02-01 00:09:35,-1,61,None,B00037,8.966667,2021/02_5,13.817206


In [15]:
df_result = df[['ride_id','prediction']]

df_result.head()

,ride_id,prediction
1,2021/02_1,14.539865
2,2021/02_2,13.740422
3,2021/02_3,15.593339
4,2021/02_4,15.188118
5,2021/02_5,13.817206


In [16]:
df_result.to_parquet(
    'prediction_tables.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)

In [17]:
!ls -l

total 19304
-rw-rw-r-- 1 david david      539 Wax 27 22:02 Dockerfile
-rw-rw-r-- 1 david david    17760 Wax 27 21:44 model.bin
-rw-rw-r-- 1 david david      163 Wax 27 22:00 Pipfile
-rw-r--r-- 1 david david     8607 Wax 27 22:00 Pipfile.lock
-rw-rw-r-- 1 david david 19711435 Wax 27 22:17 prediction_tables.parquet
-rw-rw-r-- 1 david david     7520 Wax 27 22:16 starter.ipynb
-rw-rw-r-- 1 david david     1818 Wax 27 21:54 starter.py


In [18]:
!python starter.py "2021" "03"

16.299


In [19]:
!python test_docker.py "2021" "04"

python: can't open file 'test_docker.py': [Errno 2] No such file or directory
